In [25]:
%load_ext autoreload
%autoreload 2
%pylab inline
import json

import sys
import glob
import pandas as pda
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_br import BR_DIR, make_dqn_agent
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator, TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent

from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from open_spiel.python import rl_environment
import yaml
from open_spiel.python.pytorch.ubc_rnn import AuctionRNN
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/apps/open_spiel/venv/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random', 'figure', 'solve', 'show', 'logging']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Loading BokehJS ...

In [26]:
game_name = 'big_game_2.json'
game = smart_load_sequential_game('clock_auction', dict(filename=game_name))
num_players = 2
num_products = 3
num_actions = game.num_distinct_actions()

In [27]:
input_size = len(game.new_initial_state().child(0).child(0).information_state_tensor())

In [28]:
model = AuctionRNN(2, 3, input_size, game.num_distinct_actions())

In [58]:
from open_spiel.python.pytorch import ubc_nfsp, ubc_dqn
ubc_dqn.MLP(
   input_size,
   [128,],
   num_actions,
   2, 
   3,
).reshape_infostate(x)

tensor([ 3.0000e+00,  0.0000e+00, -2.5000e-01,  1.9500e+01,  1.9250e+01,
         2.9250e+01,  2.9000e+01,  4.8750e+01,  4.8500e+01,  5.8500e+01,
         5.8250e+01,  7.8000e+01,  7.7750e+01,  0.0000e+00, -3.2750e+00,
         1.3450e+01,  1.0175e+01,  2.0175e+01,  1.6900e+01,  3.3625e+01,
         3.0350e+01,  4.0350e+01,  3.7075e+01,  5.3800e+01,  5.0525e+01,
         2.2500e+02,  2.7225e+02,  9.9825e+01,  6.6550e+01,  3.3275e+01,
         2.0000e+00,  1.0000e+00,  1.0000e+00,  4.0000e+00,  2.0000e+00,
         2.0000e+00])

In [59]:
x = game.new_initial_state().child(0).child(0).child(num_actions - 1).child(num_actions - 1).child(num_actions - 1).child(num_actions - 1).information_state_tensor()
x
# print(x[round_index(num_players)])

# x = x[2 * num_players + handcrafted_size(num_actions, num_products):]
# prefix = x[:prefix_size(num_players, num_products)]
# prefix

# x = x[prefix_size(num_players, num_products):]


[1.0,
 0.0,
 1.0,
 0.0,
 3.0,
 0.0,
 -0.25,
 19.5,
 19.25,
 29.25,
 29.0,
 48.75,
 48.5,
 58.5,
 58.25,
 78.0,
 77.75,
 0.0,
 -3.2750000953674316,
 13.449999809265137,
 10.175000190734863,
 20.174999237060547,
 16.899999618530273,
 33.625,
 30.350000381469727,
 40.349998474121094,
 37.07500076293945,
 53.79999923706055,
 50.525001525878906,
 225.0,
 272.25,
 99.82499694824219,
 66.55000305175781,
 33.275001525878906,
 2.0,
 1.0,
 1.0,
 4.0,
 2.0,
 2.0,
 1.0,
 0.0,
 300.0,
 120.0,
 80.0,
 30.0,
 2.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 4.0,
 2.0,
 2.0,
 75.0,
 50.0,
 25.0,
 2.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 4.0,
 2.0,
 2.0,
 82.5,
 55.0,
 27.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 90.75,
 60.5,
 30.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [48]:
model.reshape_infostate(x)

tensor([[  1.0000,   0.0000, 300.0000, 120.0000,  80.0000,  30.0000,   2.0000,
           1.0000,   1.0000,   2.0000,   1.0000,   1.0000,   4.0000,   2.0000,
           2.0000,  75.0000,  50.0000,  25.0000],
        [  1.0000,   0.0000, 300.0000, 120.0000,  80.0000,  30.0000,   2.0000,
           1.0000,   1.0000,   2.0000,   1.0000,   1.0000,   4.0000,   2.0000,
           2.0000,  82.5000,  55.0000,  27.5000],
        [  1.0000,   0.0000, 300.0000, 120.0000,  80.0000,  30.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,  90.7500,  60.5000,  30.2500]])

In [42]:
prefix_size(num_players, num_products) + current_round * suffix_len_per_round

30

In [43]:
current_round = int(x[round_index(num_players)])
print(current_round)
infostate_tensor = x[2 * num_players + handcrafted_size(num_actions, num_products):] # Ignore the handcrafted stuff

# split tensor into (per-auction, per-round) features
suffix_len_per_round = FEATURES_PER_PRODUCT * num_products

prefix = torch.tensor(infostate_tensor[:prefix_size(num_players, num_products)])
ub = prefix_size(num_players, num_products) + current_round * suffix_len_per_round
lb = prefix_size(num_players, num_products)
suffix = torch.tensor(infostate_tensor[lb:ub])
print(prefix)
print(suffix.reshape(-1, 4 * num_products))

3
tensor([  1.,   0., 300., 120.,  80.,  30.])
tensor([[ 2.0000,  1.0000,  1.0000,  2.0000,  1.0000,  1.0000,  4.0000,  2.0000,
          2.0000, 75.0000, 50.0000, 25.0000],
        [ 2.0000,  1.0000,  1.0000,  2.0000,  1.0000,  1.0000,  4.0000,  2.0000,
          2.0000, 82.5000, 55.0000, 27.5000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000, 90.7500, 60.5000, 30.2500]])


In [ ]:
import torch

In [ ]:
# examples for game with 2 players, 2 products, max 3 rounds
example_infostates = [
    # p1/p2  bgt  values  submitted         processed         # observed        # prices
    [ 1, 0, 1, 0, 1, 0, 100, 75, 25, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 50, 50, 60, 60, 70, 70],
    [ 1, 0, 1, 0, 1, 0, 100, 75, 25, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 50, 50, 60, 60,  0,  0],
    [ 1, 0, 1, 0, 1, 0, 100, 75, 25, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 50, 50,  0,  0,  0,  0],
]

example_tensors = [torch.tensor(i).float() for i in example_infostates]

In [ ]:
model(example_tensors)

In [ ]:
import pprofile
profiler = pprofile.Profile()
with profiler:
    model(example_tensors)

# profiler.print_stats()
profiler.dump_stats("profile.txt")

In [ ]:
torch.save(model, 'test.pt')

In [ ]:
torch.load('test.pt')

In [ ]:
import pickle
pickle.dumps(model)

In [ ]:
from open_spiel.python.pytorch import ubc_nfsp

In [ ]:
model_params = {
    'num_players': 2,
    'num_products': 2,
    'input_size': 17,
    'output_size': 4,
}
nfsp = ubc_nfsp.NFSP(0, 4, AuctionRNN, model_params, AuctionRNN, model_params, 1000, 0.1)

In [ ]:
torch.save(nfsp, 'test.pt')

In [ ]:
pickle.dumps(nfsp)

In [ ]:
def expand_infostate_tensor(infostate_tensor, num_players, num_products):
    offset = 0
    
    players = infostate_tensor[offset:offset+num_players]
    offset += num_players
    
    budget = infostate_tensor[offset:offset+1]
    offset += 1
    
    values = infostate_tensor[offset:offset+num_products]
    offset += num_products
    
    rounds = len(infostate_tensor[offset:]) // (4 * num_products)
    
    submitted_demands = infostate_tensor[offset:offset+rounds*num_products].reshape((rounds, num_products))
    offset += rounds*num_products
    
    processed_demands = infostate_tensor[offset:offset+rounds*num_products].reshape((rounds, num_products))
    offset += rounds*num_products
    
    observed_demands = infostate_tensor[offset:offset+rounds*num_products].reshape((rounds, num_products))
    offset += rounds*num_products
    
    prices = infostate_tensor[offset:offset+rounds*num_products].reshape((rounds, num_products))
    
    expanded_infostate = torch.hstack([
        torch.tile(players, (rounds, 1)),
        torch.tile(budget, (rounds, 1)),
        torch.tile(values, (rounds, 1)),
        submitted_demands,
        processed_demands,
        observed_demands, 
        prices
    ])
    return expanded_infostate

In [ ]:
infostate = torch.tensor([
    # player (one-hot encoded)
    1, 0, 0,
    # budget
    100,
    # values
    100, 50,
    # submitted demands each round
    2, 2, 
    2, 1,
    0, 0,
    # processed demands each round
    2, 2,
    2, 2,
    0, 0,
    # observed demands each round
    6, 6,
    6, 3,
    0, 0,
    # prices
    50, 50,
    60, 60,
    72, 72,
])

In [ ]:
infostate

In [ ]:
# expand into (players, budget, values, submitted demands, processed demands, observed demands, prices)
expand_infostate_tensor(infostate, num_players=3, num_products=2)